# Basic Cleaning of BG-Data

Say you just want to work with some of the data from the BG full dataset to test some hypotheses or just do some exploratory data analysis work. If that is the case, this notebook will help you accomplish that very fast, by the end of this notebook, you should have a full dataset ready for analysis.

# Packages

In [7]:
import numpy as np
import pandas as pd
from glob import glob
import re
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
import concurrent.futures as cf
import csv
csv.field_size_limit(10000000)

pd.set_option('display.max_columns', None)

%matplotlib inline

csv.field_size_limit()

10000000

In [8]:
# Where is the data at?
path_raw_data = '~/Dropbox/Burning Glass/Data/random_data/'
# Where will it go to once it's cleaned?
path_out_clean = '~/Dropbox/Burning Glass/Analysis/unsupervised_approach/'

## Columns and Data Types

I have thoroughtly examined each column in the dataset and determined that the columns below are the most useful for analysis. If you'd like to load the entire dataset with all of the columns, make sure you don't run the cell below and comment out the parameter `use_cols=` from the `read_csv()` method below.

In [9]:
best_list = ['JobID', 'CleanJobTitle', 'CanonCity', 'CanonState', 'JobDate', 'JobText', 'Source', 'CanonEmployer',
             'Latitude', 'Longitude', 'CanonIntermediary', 'CanonJobTitle', 'CanonCounty', 'DivisionCode', 'MSA', 'LMA',
             'InternshipFlag', 'ConsolidatedONET', 'CanonSkillClusters', 'CanonSkills', 'IsDuplicate', 'CanonMinimumDegree', 
             'CanonRequiredDegrees', 'CIPCode', 'MinExperience', 'ConsolidatedInferredNAICS', 'BGTOcc', 'MaxAnnualSalary',
             'MaxHourlySalary', 'MinAnnualSalary', 'MinHourlySalary', 'YearsOfExperience', 'CanonJobHours', 'CanonJobType',
             'CanonPostalCode', 'CanonYearsOfExperienceCanonLevel', 'CanonYearsOfExperienceLevel', 'ConsolidatedTitle', 
             'Language', 'BGTSubOcc', 'ConsolidatedDegreeLevels', 'MaxDegreeLevel', 'MinDegreeLevel']

## Single Data File

If you are only using 1 file, you can load it using the following code.

In [10]:
%%time

raw_data = 'rand_samp_20.csv'

df = pd.read_csv(path_raw_data + raw_data, 
                 parse_dates=['JobDate'], 
                 low_memory=False,
                 usecols=best_list
                 )
df.head()

CPU times: user 19.3 s, sys: 13.6 s, total: 32.9 s
Wall time: 38.4 s


,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
0,38807273782,Business Development Manager - Construction,Irvine,CA,2020-07-03,Business Development Manager - Construction\n\...,Job intermediary,NaN,33.7425,-117.7470,XO Staffing Agency,Construction Manager,Orange,11244.0,31080: Metropolitan Statistical Area|348: Comb...,DV064204|MT063110,0,11202200.0,Sales: Business Development;Specialized Skills...,{'Business Development': 'Sales: Business Deve...,False,Bachelor's,Bachelor's,520201|521401|522001,NaN,2362.0,11-2022.00,88000.0,42.31,63000.0,30.29,NaN,NaN,NaN,92602,NaN,NaN,Construction Manager,en,Business Development Manager,16,NaN,16.0
1,38765089544,Stihl Repair Technician,Tucson,AZ,2020-04-19,STIHL Repair Technician\n\nEquipmentLocator.co...,Job Board,Equipmentlocator Com,32.2161,-110.9710,NaN,Repair Technician,Pima,NaN,46060: Metropolitan Statistical Area,MT044606,0,49907100.0,Specialized Skills|Specialized Skills,"{'Repair': 'Specialized Skills', 'Retail Indus...",False,NaN,NaN,NaN,NaN,5241.0,49-9071.91,NaN,NaN,NaN,NaN,NaN,fulltime,permanent,85701,NaN,NaN,Repair Technician,en,Building and General Maintenance Technician,NaN,NaN,NaN
2,38726367466,Pediatrics In - Academic Pediatric Geneticist,Fresno,CA,2020-03-10,Pediatrics in CA - Academic Pediatric Genetici...,Job intermediary,NaN,36.8410,-119.8000,Comphealth,NaN,Fresno,NaN,23420: Metropolitan Statistical Area,MT062342,0,19102903.0,Science and Research: Genetics;Specialized Ski...,{'Genetics': 'Science and Research: Genetics;S...,False,NaN,NaN,NaN,NaN,62.0,19-1020.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93650,NaN,NaN,"Academic Pediatric Geneticist, Information And...",en,Biologist,NaN,NaN,NaN
3,38813841551,Ui Developer At The Judge Group,Irving,TX,2020-07-13,"UI Developer at The Judge Group\n\nIrving, TX\...",Job intermediary,NaN,32.8420,-96.9719,"The Judge Group , Incorporated",User Interface (UI) Developer,Dallas,19124.0,19100: Metropolitan Statistical Area|206: Comb...,DV481912|MT481910,0,15113400.0,Information Technology: JavaScript and jQuery;...,{'AngularJS': 'Information Technology: JavaScr...,False,NaN,NaN,NaN,8.0,54.0,15-1134.93,NaN,NaN,NaN,NaN,Min 2 Years|min 1 Year|8-10 years,fulltime,temporary,75014,6+,high,User Interface (UI) Developer,en,User Interface (UI) Developer,NaN,NaN,NaN
4,38799678091,Cdl A Regional Truck Driver,Chestertown,MD,2020-06-20,CDL A Regional Truck Driver\n\nCowan Systems\n...,Recruiter,Cowan Systems,39.2169,-76.0767,NaN,Class A CDL Truck Driver,Kent,NaN,NaN,CN240290,0,53303200.0,Administration: Scheduling;Specialized Skills,{'Scheduling': 'Administration: Scheduling;Spe...,False,NaN,NaN,NaN,3.0,484.0,53-3032.00,65000.0,31.25,54000.0,25.96,one year|three years|One year|3 years,NaN,NaN,21620,1-6,mid,Class A CDL Truck Driver,en,Regional Truck Driver,NaN,NaN,NaN


## Multiple Files

If you want to analyze multiple files, and the combined size of the files are less than your memory RAM, use the following code.

**Note**: If you want to clean many files that don't fit into memory (say 50GB), navigate to the cleaning data at scale notebook and use dask instead.

In [ ]:
files = glob('../../../../Dropbox/Burning Glass/Data/random_data/ran*.csv')
files[:3]

In [ ]:
%%time

# You will create a function to read your CSV files
def get_data(data):
    return pd.read_csv(data, low_memory=False, parse_dates=['JobDate'], usecols=small_list)

# Then you will parallelize it to all the cores in your machine
# as you read the files
with cf.ProcessPoolExecutor() as executor:
    results = executor.map(get_data, files)

# concatenate all of the results together
df = pd.concat(results)

# reset the mismatched indexes and keep the changes
df.reset_index(drop=True, inplace=True)

# check it out
df.head(3)

# Examine the Data

Now that we have our dataset ready, let's see where we are at in terms of missing values.

In [11]:
df.shape

(500000, 43)

In [12]:
df.tail()

,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel
499995,38791254071,Registered Nurse/Travel Assignment,Moreno Valley,CA,2020-06-07,Registered Nurse (RN) / Travel Assignment\n\nD...,Job intermediary,NaN,33.8885,-117.2220,Accountable Healthcare Staffing,Registered Nurse,Riverside,NaN,40140: Metropolitan Statistical Area,MT064014,0,29114100.0,Health Care: Basic Living Activities Support;S...,{'Caregiving': 'Health Care: Basic Living Acti...,False,NaN,NaN,NaN,NaN,62.0,29-1141.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92551,NaN,NaN,Registered Nurse,en,Registered Nurse,NaN,NaN,NaN
499996,38779945336,Assistant Human Resources Manager,Alpharetta,GA,2020-05-15,Assistant HR Manager\n\nFamily Flowers - Alpha...,Job Board,Family Flowers,34.1222,-84.2975,NaN,Human Resources Assistant,Fulton,NaN,12060: Metropolitan Statistical Area,MT131206,0,11312100.0,Business: Business Strategy;Specialized Skills...,{'Business Strategy': 'Business: Business Stra...,False,NaN,NaN,NaN,NaN,4531.0,11-3121.92,55000.0,26.44,40000.0,19.23,2 years,fulltime,permanent,30004,NaN,NaN,Human Resources Assistant,en,Human Resources Manager,NaN,NaN,NaN
499997,38758065731,Registered Nurse Senior Living,Grand Rapids,MI,2020-04-07,Job Details\n\n*\n\nRegistered Nurse (RN) FULL...,Job Board,Samaritas,42.9664,-85.6552,NaN,Registered Nurse,Kent,NaN,24340: Metropolitan Statistical Area,MT262434,0,29114100.0,Administration: Administrative Support;Special...,{'Appointment Setting': 'Administration: Admin...,False,Associate's,Associate's|Bachelor's,NaN,1.0,62.0,29-1141.00,NaN,NaN,NaN,NaN,One year,NaN,NaN,49501,0-1,low,Registered Nurse,en,Registered Nurse,14|16,16.0,14.0
499998,38749146523,02 Night Cook & Prep - Gulf Coast Town Center,Fort Myers,FL,2020-03-29,NaN,Job Board,Zaxby's Southwest Florida,26.6285,-81.8674,NaN,Prep Cook,Lee,NaN,15980: Metropolitan Statistical Area,MT121598,0,35201400.0,Specialized Skills,{'Multi-Tasking': 'Specialized Skills'},False,NaN,NaN,NaN,NaN,722.0,35-2011.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33901,NaN,NaN,Prep Cook,en,Cook,NaN,NaN,NaN
499999,38821741759,Patient Financial Coordinator - Certified Appl...,Des Moines,IA,2020-07-26,Patient Financial Coordinator - Certified Appl...,Education,Methodist College,41.6050,-93.6319,NaN,Financial Counselor,Polk,NaN,19780: Metropolitan Statistical Area,MT191978,0,43405103.0,Human Resources: Compensation and Benefits;Spe...,{'COBRA': 'Human Resources: Compensation and B...,False,Associate's,Associate's|Bachelor's,NaN,5.0,8131.0,43-4051.03,NaN,NaN,NaN,NaN,5 years,fulltime,permanent,50301,1-6,mid,Financial Counselor,en,Registrar / Patient Service Representative,14|16,16.0,14.0


In [13]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 43 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   JobID                             500000 non-null  int64         
 1   CleanJobTitle                     499957 non-null  object        
 2   CanonCity                         497366 non-null  object        
 3   CanonState                        499771 non-null  object        
 4   JobDate                           500000 non-null  datetime64[ns]
 5   JobText                           468540 non-null  object        
 6   Source                            494214 non-null  object        
 7   CanonEmployer                     397265 non-null  object        
 8   Latitude                          497587 non-null  float64       
 9   Longitude                         497587 non-null  float64       
 10  CanonIntermediary               

In [14]:
missing_pct = df.isna().sum() / df.shape[0] * 100
missing_pct

JobID                                0.0000
CleanJobTitle                        0.0086
CanonCity                            0.5268
CanonState                           0.0458
JobDate                              0.0000
JobText                              6.2920
Source                               1.1572
CanonEmployer                       20.5470
Latitude                             0.4826
Longitude                            0.4826
CanonIntermediary                   87.7782
CanonJobTitle                       36.3136
CanonCounty                          0.5300
DivisionCode                        68.4232
MSA                                  2.8810
LMA                                  0.8280
InternshipFlag                       0.0000
ConsolidatedONET                     3.5984
CanonSkillClusters                   5.0434
CanonSkills                          0.0000
IsDuplicate                          0.0000
CanonMinimumDegree                  49.0702
CanonRequiredDegrees            

We have lots of missing values so let's begin by dealing with the columns which we probably can't go on without, these are `JobText` and `CanonEmployer`. The first thing we will do is to filter out observations without a job posting.

In [15]:
df = df[df['JobText'].notnull()]

The next thing we'll do is to determine which samples don't have a value in the var `CanonEmployer` but do have one in `CanonIntermediary`. What we'll do is to fill in those with an intermediary with the value `"Recruiting Agency"`.

In [16]:
df.loc[(df['CanonEmployer'].isna()) & (df['CanonIntermediary'].notna()), ['CanonEmployer', 'CanonIntermediary']].head()

,CanonEmployer,CanonIntermediary
0,NaN,XO Staffing Agency
2,NaN,Comphealth
3,NaN,"The Judge Group , Incorporated"
11,NaN,Aerotek
31,NaN,Randstad


In [17]:
print(f'Missing companies BEFORE adding the Recruitment Agencies: {df.CanonEmployer.isna().sum()}')

Missing companies BEFORE adding the Recruitment Agencies: 93808


In [18]:
df.loc[(df['CanonEmployer'].isna()) & (df['CanonIntermediary'].notna()), 'CanonEmployer'] = 'Recruiting Agency'

print(f'Missing companies AFTER adding the Recruitment Agencies: {df.CanonEmployer.isna().sum()}')

Missing companies AFTER adding the Recruitment Agencies: 36897


Many jobs without an employer or recruiting agency do have a the employer in the job description. Take a look at the following example.

In [19]:
df.loc[df['CanonEmployer'].isna(), ["CleanJobTitle", "JobText"]].head()

,CleanJobTitle,JobText
23,Bartender,"Bartender\n\nposted February 8, 2020\n\nHyatt ..."
40,Security Officer - Healthcare,Security Officer - Healthcare\n\nSECURITAS -\n...
55,Hedis Coordinator,HEDIS Coordinator\n\nHealthCare Support Staffi...
57,Housekeeping Room Inspector,Housekeeping Room Inspector\n\nHEI Hotels in S...
67,"Bookkeeper, E-Commerce, Data Entry, Excel Prof...","Bookkeeper, Ecommerce, Data Entry, Excel Profe..."


In [20]:
df.loc[df['CanonEmployer'].isna(), "JobText"].iloc[0]

'Bartender\n\nposted February 8, 2020\n\nHyatt House San Diego/Sorrento Mesa\nSan Diego, CA Apply\n\nEmail Facebook Tweet LinkedIn\n\nLogo for Hyatt House San Diego/Sorrento Mesa\n\nLogo for Hyatt House San Diego/Sorrento Mesa\n\nHyatt House San Diego/Sorrento Mesa\n\n360811 l\n\nAbout This Job\n\nDescription\n\nHYATT house is an extended-stay, residential-style hotel that aims to provide individual travelers with the feel of a modern condominium. The 125 to 200 room, all-suite properties offer comforts of home such as fully equipped kitchens, flat panel HDTVs and free high-speed internet access. The public space features facilities such as a pool, a fitness center, a business center, a full breakfast every morning and the HBar in the evening. HYATT house properties are located in urban, airport and suburban locations and can accommodate small corporate meetings and corporate clients seeking to place their employees on extended assignment.\n\nBartenders are responsible for providing li

If you would like to remove observations without an employer, run the following cell. Otherwise, make sure you skip it.

In [21]:
df = df[df['CanonEmployer'].notnull()]

If you would like to add the occupations names, you can do so by running the following cells.

In [22]:
df.shape

(431643, 43)

In [23]:
%%time

# first get rid of observations without a sa
# df = df[df['BGTOcc'].notna()].copy()
df.loc[df['BGTOcc'].notnull(), 'BGTOcc'] = df.loc[df['BGTOcc'].notnull(), 'BGTOcc'].str.replace('-', '').astype(np.float32).astype(np.int32)
df['BGTOcc'].head()

CPU times: user 228 ms, sys: 28.1 ms, total: 256 ms
Wall time: 257 ms


0    112022
1    499071
2    191020
3    151134
4    533032
Name: BGTOcc, dtype: object

Load the occupations data.

In [27]:
occupations_df = pd.read_csv('~/Dropbox/Burning Glass/Analysis/occupations_clean.csv',
                             dtype={'occu_code': np.int32, 'occu_text': np.str})
occupations_df.head()

,occu_code,occu_text
0,110000,Management Occupations
1,112022,Sales Managers
2,113021,Computer and Information Systems Managers
3,113131,Training and Development Managers
4,119021,Construction Managers


In [30]:
%%time

new_df = df.merge(occupations_df, 
                  left_on='BGTOcc', 
                  right_on='occu_code', 
                  how='left')
new_df.head()

CPU times: user 1.72 s, sys: 350 ms, total: 2.07 s
Wall time: 2.16 s


,JobID,CleanJobTitle,CanonCity,CanonState,JobDate,JobText,Source,CanonEmployer,Latitude,Longitude,CanonIntermediary,CanonJobTitle,CanonCounty,DivisionCode,MSA,LMA,InternshipFlag,ConsolidatedONET,CanonSkillClusters,CanonSkills,IsDuplicate,CanonMinimumDegree,CanonRequiredDegrees,CIPCode,MinExperience,ConsolidatedInferredNAICS,BGTOcc,MaxAnnualSalary,MaxHourlySalary,MinAnnualSalary,MinHourlySalary,YearsOfExperience,CanonJobHours,CanonJobType,CanonPostalCode,CanonYearsOfExperienceCanonLevel,CanonYearsOfExperienceLevel,ConsolidatedTitle,Language,BGTSubOcc,ConsolidatedDegreeLevels,MaxDegreeLevel,MinDegreeLevel,occu_code,occu_text
0,38807273782,Business Development Manager - Construction,Irvine,CA,2020-07-03,Business Development Manager - Construction\n\...,Job intermediary,Recruiting Agency,33.7425,-117.7470,XO Staffing Agency,Construction Manager,Orange,11244.0,31080: Metropolitan Statistical Area|348: Comb...,DV064204|MT063110,0,11202200.0,Sales: Business Development;Specialized Skills...,{'Business Development': 'Sales: Business Deve...,False,Bachelor's,Bachelor's,520201|521401|522001,NaN,2362.0,112022,88000.0,42.31,63000.0,30.29,NaN,NaN,NaN,92602,NaN,NaN,Construction Manager,en,Business Development Manager,16,NaN,16.0,112022.0,Sales Managers
1,38765089544,Stihl Repair Technician,Tucson,AZ,2020-04-19,STIHL Repair Technician\n\nEquipmentLocator.co...,Job Board,Equipmentlocator Com,32.2161,-110.9710,NaN,Repair Technician,Pima,NaN,46060: Metropolitan Statistical Area,MT044606,0,49907100.0,Specialized Skills|Specialized Skills,"{'Repair': 'Specialized Skills', 'Retail Indus...",False,NaN,NaN,NaN,NaN,5241.0,499071,NaN,NaN,NaN,NaN,NaN,fulltime,permanent,85701,NaN,NaN,Repair Technician,en,Building and General Maintenance Technician,NaN,NaN,NaN,499071.0,"Maintenance and Repair Workers, General"
2,38726367466,Pediatrics In - Academic Pediatric Geneticist,Fresno,CA,2020-03-10,Pediatrics in CA - Academic Pediatric Genetici...,Job intermediary,Recruiting Agency,36.8410,-119.8000,Comphealth,NaN,Fresno,NaN,23420: Metropolitan Statistical Area,MT062342,0,19102903.0,Science and Research: Genetics;Specialized Ski...,{'Genetics': 'Science and Research: Genetics;S...,False,NaN,NaN,NaN,NaN,62.0,191020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93650,NaN,NaN,"Academic Pediatric Geneticist, Information And...",en,Biologist,NaN,NaN,NaN,NaN,NaN
3,38813841551,Ui Developer At The Judge Group,Irving,TX,2020-07-13,"UI Developer at The Judge Group\n\nIrving, TX\...",Job intermediary,Recruiting Agency,32.8420,-96.9719,"The Judge Group , Incorporated",User Interface (UI) Developer,Dallas,19124.0,19100: Metropolitan Statistical Area|206: Comb...,DV481912|MT481910,0,15113400.0,Information Technology: JavaScript and jQuery;...,{'AngularJS': 'Information Technology: JavaScr...,False,NaN,NaN,NaN,8.0,54.0,151134,NaN,NaN,NaN,NaN,Min 2 Years|min 1 Year|8-10 years,fulltime,temporary,75014,6+,high,User Interface (UI) Developer,en,User Interface (UI) Developer,NaN,NaN,NaN,151134.0,Web Developers
4,38799678091,Cdl A Regional Truck Driver,Chestertown,MD,2020-06-20,CDL A Regional Truck Driver\n\nCowan Systems\n...,Recruiter,Cowan Systems,39.2169,-76.0767,NaN,Class A CDL Truck Driver,Kent,NaN,NaN,CN240290,0,53303200.0,Administration: Scheduling;Specialized Skills,{'Scheduling': 'Administration: Scheduling;Spe...,False,NaN,NaN,NaN,3.0,484.0,533032,65000.0,31.25,54000.0,25.96,one year|three years|One year|3 years,NaN,NaN,21620,1-6,mid,Class A CDL Truck Driver,en,Regional Truck Driver,NaN,NaN,NaN,533032.0,Heavy and Tractor-Trailer Truck Drivers


In [31]:
new_df.shape

(431643, 45)